In [1]:
using Random
using Distributions
using FLoops
using Base.Threads

include("data.jl")
include("util.jl")

include("intervals/permutation.jl")
include("intervals/bootstrap.jl")
include("intervals/t.jl")

Threads.nthreads()

112

In [13]:
"""
DATA CONFIG
"""
alpha = 0.05

# data
B  = 100    # num. coverage probabilities per boxplot
S  = 1825   # num. samples per coverage probability
nx = 8      # size of group 1
ny = 8     # size of group 2
px, py = partition(nx, ny)
mc_size = try
    (binomial(nx+ny, nx) > 45_000) ? 10_000 : 0
catch
    0
end

dtype = Float32
seed = 123

"""
POPULATION SETTINGS
"""

Random.seed!(seed)

distrTypeX = Normal{dtype}
X_mu = dtype.(round.(rand(B), digits=3))
X_sigma = dtype.(round.(rand(B), digits=3))
distrX = map(distrTypeX, X_mu, X_sigma)

distrTypeY = Normal{dtype}
Y_mu = dtype.(X_mu .+ ((round.(rand(B), digits=3) * 4) .- 2))
Y_sigma = X_sigma
distrY = map(distrTypeY, Y_mu, Y_sigma)

@show distrX[1:2]
@show distrY[1:2]

deltas = @. mean(distrX) - mean(distrY);

distrX[1:2] = Normal{Float32}[Normal{Float32}(μ=0.906f0, σ=0.954f0), Normal{Float32}(μ=0.443f0, σ=0.846f0)]
distrY[1:2] = Normal{Float32}[Normal{Float32}(μ=2.766f0, σ=0.954f0), Normal{Float32}(μ=-1.273f0, σ=0.846f0)]


In [16]:
Random.seed!(123)
xs = [dtype.(rand(distrX[i], nx, S)) for i in 1:B]
ys = [dtype.(rand(distrY[i], ny, S)) for i in 1:B]
@show size(ys)
ys[2]

size(ys) = (100,)


8×1825 Matrix{Float32}:
 -1.59355   -0.459762  -0.669487   …  -1.64752    0.683074  -0.962203
  0.388304  -1.79286   -1.63423       -1.2237    -3.09633   -0.376717
 -0.425268  -1.205     -0.0510648     -1.18055   -1.57285   -1.5625
 -0.833658  -1.24474   -3.04217       -1.91764   -1.29825   -1.37089
 -1.19031   -0.212677  -1.51876       -0.700791  -0.647329  -1.37151
 -1.41831   -0.814752  -0.989404   …  -1.88047   -1.75395   -1.2411
 -1.45538   -1.79264   -2.02705       -1.00816   -0.771106  -1.79657
 -1.85007   -1.22245   -0.504659      -2.5431    -1.08917   -1.54681

In [17]:
# flatten into 3D matrix
X = reshape(hcat(xs...), nx, S, B)
Y = reshape(hcat(ys...), ny, S, B)
Y[:,:,2]

8×1825 Matrix{Float32}:
 -1.59355   -0.459762  -0.669487   …  -1.64752    0.683074  -0.962203
  0.388304  -1.79286   -1.63423       -1.2237    -3.09633   -0.376717
 -0.425268  -1.205     -0.0510648     -1.18055   -1.57285   -1.5625
 -0.833658  -1.24474   -3.04217       -1.91764   -1.29825   -1.37089
 -1.19031   -0.212677  -1.51876       -0.700791  -0.647329  -1.37151
 -1.41831   -0.814752  -0.989404   …  -1.88047   -1.75395   -1.2411
 -1.45538   -1.79264   -2.02705       -1.00816   -0.771106  -1.79657
 -1.85007   -1.22245   -0.504659      -2.5431    -1.08917   -1.54681

In [18]:
function save_permutation_results(results, B, S, dir="./")
    i = 1
    per_method = []
    for is_two_sided in [true, false]
        for pooled in [true, false]
            averages = []

            for row in eachrow(results[i,:,:])
                coverage = sum([j for (j, _) in row]) / S
                width = sum([j for (_, j) in row]) / S
                push!(averages, (coverage, width))
            end

            alpha_ = is_two_sided ? alpha : alpha / 2
            save(averages, distrX[1:B], distrY[1:B], alpha_, pooled, is_two_sided, parent_dir=dir)
            i += 1
        end
    end
end

save_permutation_results (generic function with 2 methods)

In [22]:
B = 100
S = 1825

T = Threads.nthreads()
results = Array{Union{Nothing, Tuple}, 3}(undef, 6, B, S)
fill!(results, nothing)

#@time Threads.@threads for (i,j) in collect(Iterators.product(1:B, 1:S)) # 15.52 sec on (B,S) = (5, 1800)
@time @floop ThreadedEx(basesize=div(B*S, T)) for b in 1:B, s in 1:S
    @inbounds x = X[:,s,b]
    @inbounds y = Y[:,s,b]

    results[1, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), true, alpha, twoSided, twoSided)    
    results[2, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), false, alpha, twoSided, twoSided)
    results[3, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), true, alpha/2, greater, smaller, 0.0005)
    results[4, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), false, alpha/2, greater, smaller, 0.0005)
    
    """
    lo, hi = bootstrap(x, y, 0.05, false, 10_000)
    coverages[5] += (lo <= delta <= hi)

    lo, hi = tconf(x, y, 0.05, false)
    coverages[6] += (lo <= delta <= hi)
    """
end

#save_permutation_results(results, B, S, "../results/permutation/1/")

2299.917468 seconds (1.31 G allocations: 14.534 TiB, 69.74% gc time, 0.00% compilation time)


---

In [28]:
averages = []
for row in eachrow(results[1,:,:])
    coverage = sum([j for (j, _) in row]) / S
    width = sum([j for (_, j) in row]) / S
    push!(averages, (coverage, width))
end
averages

5-element Vector{Any}:
 (0.95, 3.1827745949945476)
 (0.92, 1.6322506846556246)
 (0.85, 1.3739451044574138)
 (0.93, 0.21217739206443034)
 (0.9, 0.18690359039665994)

In [70]:
x = X[:,1,1]
y = Y[:,1,1]
@time permInterval(x, y, deltas[1], (px, py, 0), true, alpha, twoSided, twoSided)

permInterval(x, y, deltas[1], (px, py, 0), true, alpha/2, greater, smaller)

pval(x, y, 2.4218399175319254, (px, py, 0), true, twoSided)

  0.292847 seconds (510.31 k allocations: 55.944 MiB, 93.62% compilation time)


0.05034965f0